In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

df_full = pd.concat([train, test])
del train ,test

print(df_full.shape)

(19715327, 12)


In [3]:
def only_number(string):
    """reference에서 숫자인 것만 뽑아 int type으로 변환"""
    try:
        int(string)
        return int(string)
    except ValueError:
        return np.nan
    
def reference_count(df):
    """reference_int column에서 reference의 개수를 세서 새로운 df 생성"""
    multi = df.groupby(df.user_id).reference_int.value_counts() # na값 drop by default 
    df_out = pd.DataFrame(multi).rename(columns = {'reference_int':'ref_count'}).reset_index()
    return df_out

In [4]:
df_full['reference_int'] = df_full.reference.apply(only_number)

In [5]:
# ref cnt by 'user_id'
df_ref = reference_count(df_full)

In [6]:
# adj_ref df
count_mean = df_ref.groupby(['reference_int']).mean().rename(columns = {'ref_count': 'ref_count_mean'})
adj_ref = pd.merge(df_ref, count_mean, on = 'reference_int', how = 'left')
adj_ref.rename(columns = {'reference_int' : 'item_id'}, inplace = True)
adj_ref['ref_adj'] = adj_ref['ref_count'] - adj_ref['ref_count_mean']

# replace 0 ref_adj values to 1e-8 to avoid 0 denominator 
adj_ref.loc[adj_ref.ref_adj == 0, 'ref_adj'] = 1e-8

In [13]:
import time

In [17]:
time.time()

1561426669.5139935

In [28]:
def build_w_matrix(item1):
    w_matrix_column = ['item1', 'item2', 'weight']
    w_matrix = pd.DataFrame(columns = w_matrix_column)

    i = 0
    
    start_time = time.time()

        
    if i % 100 ==0:
        #end_time = time.time()
        #print('100번 돌리는데 걸린 시간(분) : ', int((end_time - start_time)/60))
        #start_time = time.time()
        print(i, "out of", len(unique_items))

    # extract all users who rated item1  
    user_data = adj_ref.loc[adj_ref.item_id == item1]
    unique_users = user_data.user_id.unique()

    # record the ratings for users who rated both item1 and item2
    record_row_columns = ['user_id', 'item1', 'item2', 'rating_adjusted_1', 'rating_adjusted_2']
    record_item_1_2 = pd.DataFrame(columns = record_row_columns)

    #for each customer who rated item1
    for c_user in unique_users:
        #print('build weight matrix for customer {0}, movie_1 {1}'.format(c_user, item1))
        # user rating for item1
        c_item1_rating = user_data.loc[user_data.user_id == c_user, 'ref_adj'].iloc[0]
        # user rating except for item1
        c_user_data = adj_ref.loc[(adj_ref.user_id == c_user)& (adj_ref.item_id != item1)]

        c_unique_movies = c_user_data.item_id.unique()

        # for each item rated by user as item2
        for item2 in c_unique_movies:

        #user rating for item2
            c_item2_rating = c_user_data.loc[c_user_data.item_id == item2, 'ref_adj'].iloc[0]
            record_row = pd.Series([c_user, item1, item2, c_item1_rating, c_item2_rating], index = record_row_columns)
            record_item_1_2 = record_item_1_2.append(record_row, ignore_index = True)

        # calculate the similarity values between item1 and the above recorded items
        unique_item2 = np.unique(record_item_1_2['item2'])

        # for each movie 2

        for item2 in unique_item2:
            #print('calculate weight for item1 {0}, item2 {1}'.format(item1, item2))
            paired_item_1_2 = record_item_1_2.loc[record_item_1_2.item2 == item2]
            sim_value_numerator = (paired_item_1_2['rating_adjusted_1'] * paired_item_1_2['rating_adjusted_2']).sum()
            sim_value_denominator = np.sqrt(np.square(paired_item_1_2['rating_adjusted_1']).sum()) * np.sqrt(np.square(paired_item_1_2['rating_adjusted_2']).sum())
            sim_value_denominator = sim_value_denominator if sim_value_denominator != 0 else 1e-8
            sim_value = sim_value_numerator / sim_value_denominator
            w_matrix = w_matrix.append(pd.Series([item1, item2, sim_value], index=w_matrix_column), ignore_index=True)
    i +=1
    return w_matrix


In [22]:
from joblib import Parallel, delayed
import multiprocessing

In [25]:
 = adj_ref.item_id.unique()

In [ ]:
num_cores = multiprocessing.cpu_count()
w_matrix = Parallel(n_jobs=num_cores)(delayed(build_w_matrix)(item1) for item1 in unique_items)


In [ ]:
w_matrix.to_csv(path+ 'w_matrix.csv', index = False)

In [23]:
# what are your inputs, and what operation do you want to 
# perform on each input. For example...
inputs = range(10) 
def processInput(i):
    return i * i
 

     
results = Parallel(n_jobs=num_cores)(delayed(processInput)(i) for i in inputs)
results

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]